In [1]:
import torch_geometric.data
from torch_geometric.utils import from_networkit
from torch_geometric import EdgeIndex
from torch_geometric.data import Data
from torch.distributions import Geometric
from torch_cluster import random_walk
import networkit as nk
import math
import torch
import random


def get_incoming_neighbors(edge_index, num_nodes):
    # Create an empty list for each node
    incoming_edges = {node: [] for node in range(num_nodes)}

    # Iterate over the edge_index to populate the incoming_edges
    for source, target in edge_index.t():
        incoming_edges[target.item()].append(source.item())

    return incoming_edges


def calculate_out_degrees(edge_index, num_nodes):
    # Initialize a tensor to hold the out-degree of each node
    out_degrees = torch.zeros(num_nodes, dtype=torch.long)

    # Count the occurrences of each node in the first row of edge_index
    for source_node in edge_index[0]:
        out_degrees[source_node] += 1

    return out_degrees


def geometric_random_walk(edge_index: EdgeIndex, num_nodes, prob_success, start_node):
    # Sample a length for the random walk from a geometric distribution
    geom_dist = Geometric(prob_success)
    walk_length = max(
        1, geom_dist.sample().round().int().item()
    )  # Convert to Python int

    # Perform the random walk
    row, col = edge_index
    # print(walk_length)
    walk = random_walk(
        row=row,
        col=col,
        start=torch.tensor([start_node], dtype=torch.long),
        walk_length=walk_length,
    )
    assert walk.shape[0] == 1
    return walk[0].tolist()


class PPR:
    def __init__(self, G: EdgeIndex, alpha, delta):
        self.alpha = alpha
        self.delta = delta
        self.c = 350
        self.beta = 1 / 6
        self.e_rev = math.sqrt(delta)

        self.g_edge_index = G
        assert self.g_edge_index.num_cols == self.g_edge_index.num_rows
        self.num_nodes = self.g_edge_index.num_cols
        self.incoming_neighbors = get_incoming_neighbors(
            self.g_edge_index, self.num_nodes
        )
        self.out_degrees = calculate_out_degrees(self.g_edge_index, self.num_nodes)

    def fast_ppr(self, s, t):
        t_set, f_set, pi_inv = self.frontier(t)

        if s in t_set:
            return pi_inv[s]

        target_num_walks = int(self.c * self.e_rev / self.delta)
        # print(target_num_walks)
        hit_nodes = []
        for _ in range(target_num_walks):
            walk = geometric_random_walk(
                self.g_edge_index, self.num_nodes, self.alpha, s
            )
            for n in walk:
                if n in f_set:
                    hit_nodes.append(n)
                    break

        result = torch.Tensor([0.0])
        for n in hit_nodes:
            result += pi_inv[n]
        result /= target_num_walks

        return result.item(), len(t_set) + len(f_set)
        ...

    def frontier(self, t):
        e_inv = self.beta * self.e_rev

        estimate_vector = torch.zeros([self.num_nodes])
        estimate_vector[t] = self.alpha
        residual_vector = estimate_vector.clone()

        target_set = {t}
        frontier_set = set(self.incoming_neighbors[t])

        while True:
            w = torch.argmax(residual_vector).item()
            if residual_vector[w] <= self.alpha * e_inv:
                break
            for u in self.incoming_neighbors[w]:

                big_d = (1 - self.alpha) * residual_vector[w] / self.out_degrees[u]
                estimate_vector[u] += big_d
                residual_vector[u] += big_d
                if estimate_vector[u] > self.e_rev:
                    target_set.add(u)
                    frontier_set.update(self.incoming_neighbors[u])
            residual_vector[w] = 0

        frontier_set = frontier_set - target_set

        return target_set, frontier_set, residual_vector

In [3]:
G_nk = nk.readGraph("./foodweb-baydry.konect")
n = G_nk.numberOfNodes()
is_undir = not G_nk.isDirected()
G_pyg = from_networkit(G_nk)[0]
G_pyg = EdgeIndex(G_pyg, sparse_size=(n, n), is_undirected=is_undir)

In [4]:
def run_for_alpha(alpha):

    print("--------------------------------")

    delta = 1 / n
    print("delta:", delta)
    print("alpha:", alpha)

    my_ppr = PPR(G_pyg, alpha=alpha, delta=delta)

    random.seed(0)
    num_node_pairs = 20
    num_below_delta = 0
    num_non_zero = 0
    average_size_target_set = 0
    for _ in range(num_node_pairs):
        # print("----------------")
        s = random.randint(0, n - 1)
        t = s
        while t == s:
            t = random.randint(0, n - 1)

        # print(s, t)
        score, target_set_size = my_ppr.fast_ppr(s, t)
        # print(score, target_set_size)
        if score > 0:
            num_non_zero += 1
        if score < delta:
            num_below_delta += 1
        average_size_target_set += target_set_size
    average_size_target_set /= num_node_pairs

    print("below delta:", int(100 * num_below_delta / num_node_pairs ), "%")
    print("non zero:", int(100 * num_non_zero / num_node_pairs ), "%")
    print("average size target set:", average_size_target_set)

In [29]:
run_for_alpha(0.0001)

--------------------------------
delta: 0.0078125
alpha: 0.0001
below delta: 100 %
non zero: 70 %
average size target set: 16.0


In [30]:
run_for_alpha(0.1)

--------------------------------
delta: 0.0078125
alpha: 0.1
below delta: 100 %
non zero: 65 %
average size target set: 16.65


In [31]:
run_for_alpha(0.3)

--------------------------------
delta: 0.0078125
alpha: 0.3
below delta: 100 %
non zero: 65 %
average size target set: 16.65


In [32]:
run_for_alpha(0.99)

--------------------------------
delta: 0.0078125
alpha: 0.99
below delta: 100 %
non zero: 35 %
average size target set: 16.0


I think that we may have a bug in the code, since during the frontier calculation rarely any node except `t` itself is added to the target set.

This is the same for all kinds of values of `alpha`, meaning that the only impact `alpha` currently has is how long the random walks go on, so it is not surprising that with low values of alpha the non zero scores are higher and the running time is longer.

The average size of the frontier+target set is between 16 and 17 (again very similar for all values of alpha). It is important to note though, that we added the neighbors of `t` to the frontier set as long as they weren't contained in the target set. This is because otherwise (possibly because of our bug), the frontier set would be empty most of the time.

